In [1]:
BUCKET = 'food-221614'
PROJECT = 'food-221614'
REGION = 'us-central1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.8'

In [3]:
%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [4]:
%bash
gsutil ls gs://${BUCKET}/food_model

gs://food-221614/food_model/__init__.py
gs://food-221614/food_model/food_notfood.py
gs://food-221614/food_model/setup.py
gs://food-221614/food_model/food_notfood_trained/
gs://food-221614/food_model/test/
gs://food-221614/food_model/train/
gs://food-221614/food_model/valid/


In [15]:
%bash
grep "^def" food_model/food_notfood.py

def imgs_input_fn(path, mode, batch_size=15):
def cnn_model_fn(features, labels, mode):


In [71]:
%writefile food_model/food_notfood.py
import numpy as np
import os
import tensorflow as tf 
#import google.datalab.storage as storage
from google.cloud import storage

tf.logging.set_verbosity(tf.logging.INFO)
       
#Data reading and preprocessing function
def imgs_input_fn(data, mode, batch_size=15):
    
    imagepaths, imagelabels = [], []
    buffer_size = 0
   
    path = os.path.join('gs://food-221614/food_model/',data)
    
    for file_path in tf.gfile.Glob(os.path.join(path,'food')):
        imagepaths.append(file_path)
        imagelabels.append(0)  
        buffer_size += 1
    for file_path in tf.gfile.Glob(os.path.join(path,'not food')):
        imagepaths.append(file_path)
        imagelabels.append(1)
        buffer_size += 1
    
    imagepaths = tf.constant(imagepaths)
    imagelabels = tf.constant(imagelabels)
    
    def _parse_function(file, label):
        image_string = tf.read_file(file)
        image_decoded = tf.image.decode_jpeg(image_string)
        image_resized = tf.image.resize_images(image_decoded,[244,244])
        image_resized.set_shape([244, 244, 3])
        
        return image_resized, label
    
    def augment(image, label):
        augment_image = tf.random_crop(image, [244,244,3])
        augment_image = tf.image.random_flip_left_right(image)
        augment_image = tf.contrib.image.rotate(augment_image, 50)
        
        return augment_image, label
        
    dataset = tf.data.Dataset.from_tensor_slices((imagepaths,imagelabels))
    dataset = dataset.map(_parse_function)
    dataset = dataset.shuffle(buffer_size=buffer_size)
    
    #Create batches 
    if mode == tf.estimator.ModeKeys.TRAIN:
        num_epochs = None
        dataset = dataset.map(augment)
        dataset = dataset.repeat(num_epochs).shuffle(buffer_size=buffer_size)
    else:
        num_epochs = 1
    
    dataset = dataset.repeat(num_epochs).batch(batch_size)    
    iterator = dataset.make_one_shot_iterator() 
    features, labels = iterator.get_next()
    labels = tf.reshape(labels,[-1, 1])
    
    return features, labels

#def reset_graph(seed=42):
 #   tf.reset_default_graph()
  #  tf.set_random_seed(2)
   # np.random.seed(2)

#reset_graph()

def serving_input_fn():
    feature_placeholders = {'image_path':tf.placeholder(tf.string, shape=())}
    path = feature_placeholders['image_path']
    image_string = tf.read_file(filepath)
    image_decoded = tf.image.decode_jpeg(image_string)
    image_resized = tf.image.resize_images(image_decoded,[244,244])
    image = image_resized.set_shape([244, 244, 3])
    
    return tf.estimator.export.ServingInputReceiver(image, feature_placeholders)

def cnn_model_fn(features, labels, mode):
    
    input_layer = tf.reshape(features, [-1 , 244, 244, 3])
    
    #Convolutional and Pooling layers
    conv1 = tf.layers.conv2d(input_layer, 16, 3, activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(conv1, 2, 2)

    conv2 = tf.layers.conv2d(pool1, 32, 3, activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(conv2, 2, 2)

    conv3 = tf.layers.conv2d(pool2, 64, 3, activation=tf.nn.relu)
    pool3 = tf.layers.max_pooling2d(conv3, 2, 2)
    
    #Dense layer
    flat = tf.contrib.layers.flatten(pool3)
    dense = tf.layers.dense(flat, 64, activation=tf.nn.relu)
    drop = tf.layers.dropout(dense, rate=0.3, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    #Logits layer
    logits = tf.layers.dense(drop, 2)
    
    #Generate predictions
    predictions = {"classes": tf.argmax(input=logits, axis=1),"probabilities": tf.nn.softmax(logits, name="softmax_tensor")}
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        export_outputs = {'prediction': tf.estimator.export.PredictOutput(predictions)}
        
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions, export_outputs=export_outputs)
    
    #Calculate Loss
    loss = tf.losses.sparse_softmax_cross_entropy(labels, logits)
    
    #Configure the Training Op
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op) 
    
    #Add evaluation metrics
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


#food_classifier = tf.keras.estimator.model_to_estimator(keras_model=model, model_dir='food_model')
eval_int = 50

food_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir='gs://food-221614/food_model/food_notfood_trained',
                                        config=tf.estimator.RunConfig(save_checkpoints_secs=eval_int))  

train_spec = tf.estimator.TrainSpec(input_fn= lambda: imgs_input_fn('train', mode=tf.estimator.ModeKeys.TRAIN), max_steps=60)

exporter = tf.estimator.LatestExporter('exporter', serving_input_fn, exports_to_keep=None)

eval_spec = tf.estimator.EvalSpec(input_fn= lambda: imgs_input_fn('test', mode=tf.estimator.ModeKeys.EVAL),exporters = exporter,
                                   start_delay_secs=eval_int, throttle_secs=eval_int)
    
tf.estimator.train_and_evaluate(food_classifier, train_spec, eval_spec)


Overwriting food_model/food_notfood.py


In [72]:
%bash
echo "bucket=${BUCKET}"
rm -rf food-model/food_notfood_trained
export PYTHONPATH=${PYTHONPATH}:${PWD}/food_model
python -m food_model.food_notfood \
  --bucket=${BUCKET} \
  --output_dir=food_model/food_notfood_trained \
  --job-dir=./tmp \
  # --train_examples=1 --eval_steps=1

bucket=food-221614
gs://food-221614/food_model/test/food/00E8CF02-191E-4FA1-9236-CFA950CE0FD4.jpg
gs://food-221614/food_model/test/food/00FFCCD4-16FE-4EF2-8EF8-12577B8B9AF9.jpg
gs://food-221614/food_model/test/food/0A03CB4C-05AF-438E-B785-8DFCFFB71E30.jpg
gs://food-221614/food_model/test/food/0A788C68-5529-44C4-8466-76C297E7C5ED.jpg
gs://food-221614/food_model/test/food/0B71F404-ABA3-411D-91E9-588DC79C0034.jpg
gs://food-221614/food_model/test/food/0C6F203C-B1D0-4A55-A6DE-DF24C4A91B4B.jpg
gs://food-221614/food_model/test/food/0CC38F84-CE8E-4296-88C3-F60F128EE9A1.jpg
gs://food-221614/food_model/test/food/0FE2EEA2-26CD-4949-9BFB-E286872F392A.jpg
gs://food-221614/food_model/test/food/1A77C188-BC7F-4338-9708-2514D7084291.jpg
gs://food-221614/food_model/test/food/1AD0E884-F587-478D-97BC-FBC2BD636C67.jpg
gs://food-221614/food_model/test/food/1B02DAED-F9C2-4CE4-BC3D-6726FC3F9690.jpg
gs://food-221614/food_model/test/food/1B791F0E-4B62-4C99-83D8-91216F0C0988.jpg
gs://food-221614/food_model/test/

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_task_type': 'worker', '_num_worker_replicas': 1, '_train_distribute': None, '_is_chief': True, '_task_id': 0, '_num_ps_replicas': 0, '_global_id_in_cluster': 0, '_model_dir': 'gs://food-221614/food_model/food_notfood_trained', '_service': None, '_save_checkpoints_steps': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f682f14e780>, '_save_checkpoints_secs': 50, '_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_evaluation_master': '', '_keep_checkpoint_max': 5, '_master': '', '_tf_random_seed': None}
INFO:tensorflow:Running trainin

In [73]:
%bash
OUTDIR=gs://${BUCKET}/food_model/food_notfood_trained
JOBNAME=food_model/food_notfood_trained_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
  --region=$REGION \
  --module-name=food_model.food_notfood \
  --package-path=$(pwd)/food_model \
  --job-dir=$OUTDIR \
  --staging-bucket=gs://$BUCKET \
  --scale-tier=STANDARD_1 \
  --runtime-version=$TFVERSION \
  #-- \
  #--bucket=${BUCKET} \
  #--output_dir=${OUTDIR} \
 

gs://food-221614/food_model/food_notfood_trained us-central1 food_model_181106_231843
jobId: food_model_181106_231843
state: QUEUED


Removing gs://food-221614/food_model/food_notfood_trained/#1541544126976754...
Removing gs://food-221614/food_model/food_notfood_trained/checkpoint#1541544128396623...
Removing gs://food-221614/food_model/food_notfood_trained/eval/#1541543765443466...
Removing gs://food-221614/food_model/food_notfood_trained/eval/events.out.tfevents.1541543765.2b69de6db80b#1541544137934340...
Removing gs://food-221614/food_model/food_notfood_trained/events.out.tfevents.1541543676.2b69de6db80b#1541544129838238...
Removing gs://food-221614/food_model/food_notfood_trained/graph.pbtxt#1541544095672852...
Removing gs://food-221614/food_model/food_notfood_trained/model.ckpt-45.data-00000-of-00001#1541544020115598...
Removing gs://food-221614/food_model/food_notfood_trained/model.ckpt-45.index#1541544020393815...
Removing gs://food-221614/food_model/food_notfood_trained/model.ckpt-45.meta#1541544022115959...
Removing gs://food-221614/food_model/food_notfood_trained/model.ckpt-46.data-00000-of-00001#1541544052

In [ ]:
TensorBoard().start('gs://food-221614/food_model/food_notfood_trained')

In [ ]:
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)
  print('Stopped TensorBoard with pid {}'.format(pid))